In [ ]:
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense

In [ ]:
def load_data_from_mat(file_path, key):
    data = loadmat(file_path)
    return data[key]

In [ ]:
# Define paths and keys for each class
file_info_dict = {
    'A': [('/content/drive/MyDrive/emdvmd/EMDF.mat', 'IMFemd1F')],
    'B': [('/content/drive/MyDrive/emdvmd/EMDN.mat', 'IMFemd1N')],
    'C': [('/content/drive/MyDrive/emdvmd/EMDO.mat', 'IMFemd1O')],
    'D': [('/content/drive/MyDrive/emdvmd/EMDS.mat', 'IMFemd1S')],
    'E': [('/content/drive/MyDrive/emdvmd/EMDZ.mat', 'IMFemd1Z')]
}

In [ ]:
def prepare_data_for_classification(classes):
    X, y = [], []
    for label, class_id in enumerate(classes):
        file_info = file_info_dict[class_id]  # Get file paths and keys for each class
        for file_path, key in file_info:
            data = load_data_from_mat(file_path, key)
            X.append(data)
            y.append(np.array([label] * data.shape[0]))  # Assign the same label to all samples
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y

In [ ]:
# Define model creation function
def create_cnn_lstm_model(input_shape, filters, kernel_size, pool_size, lstm_units1, lstm_units2, output_units):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(units=lstm_units1, return_sequences=True))
    model.add(LSTM(units=lstm_units2))
    model.add(Dense(units=output_units, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def compute_metrics(y_true, y_pred):
    y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average='macro')
    specificity = recall_score(y_true, y_pred, average='macro', labels=[0, 1])
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, sensitivity, specificity, f1

In [ ]:
def train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, filters, kernel_size, pool_size, lstm_units1, lstm_units2, batch_size, epochs):
    model = create_cnn_lstm_model((X_train.shape[1], 1), filters, kernel_size, pool_size, lstm_units1, lstm_units2, y_train.shape[1])
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=1)
    y_pred = model.predict(X_test)
    score = model.evaluate(X_test, y_test, verbose=1)
    accuracy, sensitivity, specificity, f1 = compute_metrics(y_test, y_pred)
    print(f'Test loss: {score[0]}')
    print(f'Test accuracy: {accuracy}')
    print(f'Sensitivity (Recall): {sensitivity}')
    print(f'Specificity: {specificity}')
    print(f'F1 Score: {f1}')

# Classification A-B vs. C-D-E

In [ ]:
# Classification A-B vs. C-D-E
# Load and preprocess data for classes A-B vs. C-D-E
classes_ab = ['A', 'B']
classes_cde = ['C', 'D', 'E']

X_ab, y_ab = prepare_data_for_classification(classes_ab)
X_cde, y_cde = prepare_data_for_classification(classes_cde)

# Merge and create new labels
X = np.concatenate([X_ab, X_cde])
y = np.concatenate([np.zeros(len(y_ab)), np.ones(len(y_cde))])  # 0 for A-B, 1 for C-D-E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN-LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
y = y.astype(int)  # Ensure labels are integers
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the CNN-LSTM model
train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, filters=64, kernel_size=3, pool_size=2, lstm_units1=128, lstm_units2=128, batch_size=32, epochs=20)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 91s 196ms/step - accuracy: 0.8557 - loss: 0.2918 - val_accuracy: 0.9701 - val_loss: 0.0929
Epoch 2/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 80s 193ms/step - accuracy: 0.9697 - loss: 0.0829 - val_accuracy: 0.9823 - val_loss: 0.0392
Epoch 3/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 82s 193ms/step - accuracy: 0.9822 - loss: 0.0506 - val_accuracy: 0.9893 - val_loss: 0.0290
Epoch 4/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 83s 196ms/step - accuracy: 0.9846 - loss: 0.0426 - val_accuracy: 0.9640 - val_loss: 0.1122
Epoch 5/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 80s 192ms/step - accuracy: 0.9881 - loss: 0.0391 - val_accuracy: 0.9915 - val_loss: 0.0226
Epoch 6/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 88s 207ms/step - accuracy: 0.9903 - loss: 0.0301 - val_accuracy: 0.9927 - val_loss: 0.0187
Epoch 7/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 139s 200ms/step - accuracy: 0.9892 - loss: 0.0284 - val_accuracy: 0.9863 - val_loss: 0.0385
Epoch 8/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 80s 196ms/step - accuracy: 0.9915 - loss: 

# Classification A vs. E

In [ ]:
# Load and preprocess data for classes A and E
classes_a = ['A']
classes_e = ['E']
X_a, y_a = prepare_data_for_classification(classes_a)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_a, X_e])
y = np.concatenate([np.zeros(len(y_a)), np.ones(len(y_e))])  # 0 for A, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN-LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the CNN-LSTM model
train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, filters=64, kernel_size=3, pool_size=2, lstm_units1=128, lstm_units2=128, batch_size=32, epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


164/164 ━━━━━━━━━━━━━━━━━━━━ 35s 182ms/step - accuracy: 0.7728 - loss: 0.4117 - val_accuracy: 0.9741 - val_loss: 0.0949
Epoch 2/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 40s 175ms/step - accuracy: 0.9602 - loss: 0.1058 - val_accuracy: 0.9733 - val_loss: 0.0869
Epoch 3/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 30s 183ms/step - accuracy: 0.9705 - loss: 0.0784 - val_accuracy: 0.9832 - val_loss: 0.0440
Epoch 4/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 40s 176ms/step - accuracy: 0.9881 - loss: 0.0335 - val_accuracy: 0.9786 - val_loss: 0.0649
Epoch 5/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 42s 184ms/step - accuracy: 0.9859 - loss: 0.0409 - val_accuracy: 0.9870 - val_loss: 0.0369
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.9891 - loss: 0.0339
Test loss: 0.023321976885199547
Test accuracy: 0.9920683343502136
Sensitivity (Recall): 0.9921397282317183
Specificity: 0.9921397282317183
F1 Score: 0.9920645059170772


# Classification B vs. E

In [ ]:
# Load and preprocess data for classes B and E
classes_b = ['B']
classes_e = ['E']
X_b, y_b = prepare_data_for_classification(classes_b)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_b, X_e])
y = np.concatenate([np.zeros(len(y_b)), np.ones(len(y_e))])  # 0 for B, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN-LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the CNN-LSTM model
train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, filters=64, kernel_size=3, pool_size=2, lstm_units1=128, lstm_units2=128, batch_size=32, epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


164/164 ━━━━━━━━━━━━━━━━━━━━ 39s 214ms/step - accuracy: 0.8363 - loss: 0.3141 - val_accuracy: 0.9779 - val_loss: 0.0567
Epoch 2/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 33s 201ms/step - accuracy: 0.9777 - loss: 0.0662 - val_accuracy: 0.9863 - val_loss: 0.0517
Epoch 3/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 198ms/step - accuracy: 0.9835 - loss: 0.0513 - val_accuracy: 0.9901 - val_loss: 0.0323
Epoch 4/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 40s 194ms/step - accuracy: 0.9901 - loss: 0.0351 - val_accuracy: 0.9863 - val_loss: 0.0360
Epoch 5/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 44s 209ms/step - accuracy: 0.9896 - loss: 0.0347 - val_accuracy: 0.9962 - val_loss: 0.0129
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.9888 - loss: 0.0308
Test loss: 0.029943004250526428
Test accuracy: 0.9890176937156803
Sensitivity (Recall): 0.9892857142857143
Specificity: 0.9892857142857143
F1 Score: 0.9890155306144333


# Classification B vs. D

In [ ]:

# Load and preprocess data for classes B and D
classes_bd = ['B', 'D']
X, y = prepare_data_for_classification(classes_bd)

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN-LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the CNN-LSTM model
train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, filters=64, kernel_size=3, pool_size=2, lstm_units1=128, lstm_units2=128, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


164/164 ━━━━━━━━━━━━━━━━━━━━ 33s 178ms/step - accuracy: 0.8936 - loss: 0.1600 - val_accuracy: 1.0000 - val_loss: 0.0040
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 28s 173ms/step - accuracy: 0.9999 - loss: 0.0021 - val_accuracy: 1.0000 - val_loss: 4.3247e-04
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 173ms/step - accuracy: 1.0000 - loss: 3.5889e-04 - val_accuracy: 1.0000 - val_loss: 1.9385e-04
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 29s 176ms/step - accuracy: 1.0000 - loss: 1.6928e-04 - val_accuracy: 1.0000 - val_loss: 1.1362e-04
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 29s 180ms/step - accuracy: 1.0000 - loss: 1.0605e-04 - val_accuracy: 1.0000 - val_loss: 7.5848e-05
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 42s 183ms/step - accuracy: 1.0000 - loss: 7.2095e-05 - val_accuracy: 1.0000 - val_loss: 5.4260e-05
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 29s 175ms/step - accuracy: 1.0000 - loss: 5.1862e-05 - val_accuracy: 1.0000 - val_loss: 4.0537e-05
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 40s 167ms/

# Classification A vs. C

In [ ]:
# Load and preprocess data for classes A and C
classes_ac = ['A']
classes_c = ['C']
X_a, y_a = prepare_data_for_classification(classes_ac)
X_c, y_c = prepare_data_for_classification(classes_c)

# Merge and create new labels
X = np.concatenate([X_a, X_c])
y = np.concatenate([np.zeros(len(y_a)), np.ones(len(y_c))])  # 0 for A, 1 for C

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN-LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the CNN-LSTM model
train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, filters=64, kernel_size=3, pool_size=2, lstm_units1=128, lstm_units2=128, batch_size=32, epochs=10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 37s 196ms/step - accuracy: 0.8729 - loss: 0.2246 - val_accuracy: 0.9931 - val_loss: 0.0209
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 44s 211ms/step - accuracy: 0.9921 - loss: 0.0265 - val_accuracy: 0.9954 - val_loss: 0.0121
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 38s 193ms/step - accuracy: 0.9959 - loss: 0.0131 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 32s 195ms/step - accuracy: 0.9979 - loss: 0.0089 - val_accuracy: 0.9962 - val_loss: 0.0117
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 194ms/step - accuracy: 0.9960 - loss: 0.0159 - val_accuracy: 1.0000 - val_loss: 5.8139e-04
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 32s 193ms/step - accuracy: 0.9996 - loss: 0.0011 - val_accuracy: 0.9977 - val_loss: 0.0040
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 191ms/step - accuracy: 0.9992 - loss: 0.0039 - val_accuracy: 0.9977 - val_loss: 0.0039
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 43s 201ms/step - accuracy: 0.9990 - los

# Classification C vs. E

In [ ]:
# Load and preprocess data for classes C and E
classes_c = ['C']
classes_e = ['E']
X_c, y_c = prepare_data_for_classification(classes_c)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_c, X_e])
y = np.concatenate([np.zeros(len(y_c)), np.ones(len(y_e))])  # 0 for C, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN-LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the CNN-LSTM model
train_and_evaluate_cnn_lstm_model(X_train, y_train, X_test, y_test, filters=64, kernel_size=3, pool_size=2, lstm_units1=128, lstm_units2=128, batch_size=32, epochs=10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 39s 204ms/step - accuracy: 0.8503 - loss: 0.2324 - val_accuracy: 0.9878 - val_loss: 0.0355
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 33s 200ms/step - accuracy: 0.9935 - loss: 0.0237 - val_accuracy: 0.9985 - val_loss: 0.0061
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 202ms/step - accuracy: 0.9939 - loss: 0.0196 - val_accuracy: 0.9977 - val_loss: 0.0047
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 42s 208ms/step - accuracy: 0.9964 - loss: 0.0123 - val_accuracy: 0.9817 - val_loss: 0.0740
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 39s 198ms/step - accuracy: 0.9931 - loss: 0.0228 - val_accuracy: 0.9985 - val_loss: 0.0065
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 196ms/step - accuracy: 0.9939 - loss: 0.0188 - val_accuracy: 0.9954 - val_loss: 0.0141
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 195ms/step - accuracy: 0.9956 - loss: 0.0202 - val_accuracy: 0.9901 - val_loss: 0.0277
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 42s 200ms/step - accuracy: 0.9904 - loss: 0